In [1]:
import pyspark

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col,slice
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import rand
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline

In [3]:
spark = SparkSession.builder.appName('app_name').getOrCreate()
#master('local[*]').config('spark.sql.execution.arrow.pyspark.enabled', True).config('spark.sql.session.timeZone', 'UTC').config('spark.driver.memory','32G').config('spark.ui.showConsoleProgress', True).config('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
spark

In [4]:
#dataframe=spark.read.option('header','true','inferSchema','true').csv('02-14-2018.csv')
dataframe = spark.read.csv("02-14-2018.csv", header=True, inferSchema=True)

In [5]:
dataframe.count()

1048575

In [6]:
dataframe.printSchema()

root
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: long (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: integer (nullable = true)
 |-- TotLen Bwd Pkts: integer (nullable = true)
 |-- Fwd Pkt Len Max: integer (nullable = true)
 |-- Fwd Pkt Len Min: integer (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: integer (nullable = true)
 |-- Bwd Pkt Len Min: integer (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: long (nullable = true)
 |-- Flow IAT Min: long (nullable = true)
 |

In [7]:
nullColumns = []
for k in dataframe.columns:
    nullRows = dataframe.where(col(k).isNull()).count()
    nullR=dataframe.where(isnan(k)).count()
    if nullR>0:
        print(nullR)
    if nullRows>0 or nullR>0:
        nullColumns.append(k)
print(nullColumns)
print(len(nullColumns))
 

2277
['Flow Byts/s']
1


In [ ]:
(train, test) = dataframe.randomSplit([0.8, 0.2])

xtrain = train.select(train.columns[0:-1])
ytrain = train.select(train.columns[-1])
xtest = test.select(test.columns[0:-1])
ytest = test.select(test.columns[-1])


In [10]:
# parameter
fold = {'xt':xtrain, 'yt':ytrain, 'xv':xtest, 'yv':ytest}
k    = 5     # k-value in KNN
N    = 10    # number of particles
T    = 100   # maximum number of iterations
opts = {'k':k, 'fold':fold, 'N':N, 'T':T}

In [41]:
#lb - lower bound, ub - upper bound, N - no. of particles, dim - no of columns
def init_position(lb, ub, N, dim):
    #creates an array of N rows and dim column matrix of type float with value 0.0
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            #We add a random value to each element in matrix using initial population technique
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()        
    return X

In [40]:
lb = 0
ub = 1
dim = np.size(ytrain)
if np.size(lb) == 1:
    ub = ub * np.ones([1, dim], dtype='float')
    lb = lb * np.ones([1, dim], dtype='float')

print(dim)

1


In [42]:
init_position(lb, ub, N, dim)

array([[0.13000555],
       [0.74991725],
       [0.38250466],
       [0.0479645 ],
       [0.90346516],
       [0.42636365],
       [0.24419776],
       [0.85911927],
       [0.91207898],
       [0.73076261]])

In [1]:
#X- init array, thres - threshold to convert to 1 or 0
def binary_conversion(X, thres, N, dim):
    #creates new matrix
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            #depending on previous(init) matrix it will add 1 or 0 in Xbin
            if X[i,d] > thres:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0
    return Xbin